### Human GEM Model KO Testing

Testing loading of Human GEM model for KO

In [1]:
from Compass.compass import models, compass, utils
import importlib
import pandas as pd
import os
import cplex

In [2]:
from Compass.compass.models import MetabolicModel

In [3]:
# Making changes in the local git repo - reload the .py file 
importlib.reload(models)
importlib.reload(models.importSBML3)
importlib.reload(models.geneSymbols)
importlib.reload(models.importCommon)

<module 'Compass.compass.models.importCommon' from '/home/dzhang/Compass_Integration/Compass/compass/models/importCommon.py'>

#### Comparing dropout and non-dropout KO models

Make sure to switch branches for dropout and non_dropout models 

In [4]:
# Check that there are no reactions dropped out 
dropout_model, rxns = models.load_metabolic_model("HumanGEM", None, knockout = ['SMPD1'], knockout_type = 'flux') 

In [28]:
rxns

[]

Note that in between these two function calls, I change the Compass code to the branch with KO reactions dropped out

In [29]:
no_dropout_model, rxns = models.load_metabolic_model("HumanGEM", None, knockout = ['SMPD1'], knockout_type = 'flux') 

In [30]:
rxns

[]

In [39]:
# SMPD1 
no_dropout_model = models.init_model("HumanGEM", "homo_sapiens", 1.0, knockout = ['SMDP1'], knockout_type = 'flux')

In [62]:
dropout_model = models.init_model("HumanGEM", "homo_sapiens", 1.0, knockout = ['SMDP1'], knockout_type = 'flux')

In [54]:
dropout_model.reactions['MAR03905_pos'].meta

{}

In [71]:
def check_reaction(reaction1, reaction2):
    if reaction.name != nd_reaction.name:
        return False
    elif reaction.subsystem != nd_reaction.subsystem: 
        return False
    elif reaction.upper_bound != nd_reaction.upper_bound:
        return False
    elif reaction.lower_bound != nd_reaction.lower_bound:
        return False
    elif reaction.reactants != nd_reaction.reactants:
        return False
    elif reaction.products != nd_reaction.products:
        return False
    elif reaction.list_genes() != nd_reaction.list_genes():
        return False
    return True 

In [73]:
for id, reaction in dropout_model.reactions.items():
    nd_reaction = no_dropout_model.reactions[id]
    # Check for subsystem, name, upper/lower bound, reactants, products, list genes, reverse reaction 
    if not check_reaction(reaction, nd_reaction):
        print(id)
    if not check_reaction(reaction.reverse_reaction, nd_reaction.reverse_reaction):
        print(id)

### Check r_max for reactions with bounds = 0 in KO models

In [80]:
model, KO_rxns = models.load_metabolic_model("HumanGEM", None, ['MVK'], "flux", "homo_sapiens")

In [81]:
KO_rxns

['MAR01445', 'MAR01520']

In [82]:
# Check that the bounds are not null
for rxn in KO_rxns:
    print(model.reactions[rxn].upper_bound)
    print(model.reactions[rxn].lower_bound)

1000
0.0
1000
0.0


In [86]:
model = models.init_model("HumanGEM", "homo_sapiens", 1.0, knockout = ['MVK'], knockout_type = 'flux')

In [88]:
# Check that the bounds are 0
for rxn in KO_rxns:
    print(model.reactions[rxn + "_pos"].upper_bound)
    print(model.reactions[rxn + "_pos"].lower_bound)

0
0
0
0


In [96]:
# Obtain reactions 
reactions = []
for rxn in KO_rxns:
    reactions.append(model.reactions[rxn + "_pos"])
reactions

In [102]:
# Check r_max for both reactions 
problem = compass.algorithm.initialize_cplex_problem(model, 1, 4, 2)

In [103]:
for reaction in reactions:
    partner_reaction = reaction.reverse_reaction

    # Set partner reaction upper-limit to 0 in problem
    # Store old limit for later to restore
    if partner_reaction is not None:
        partner_id = partner_reaction.id
        old_partner_ub = problem.variables.get_upper_bounds(partner_id)
        old_partner_lb = problem.variables.get_lower_bounds(partner_id)
        problem.variables.set_upper_bounds(partner_id, max(old_partner_lb, 0))

    utils.reset_objective(problem)
    problem.objective.set_linear(reaction.id, 1.0)
    problem.objective.set_name(str(reaction.id))
    problem.objective.set_sense(problem.objective.sense.maximize)

    problem.solve()

    rxn_max = problem.solution.get_objective_value()

    if rxn_max == 0:
        print("ok")

ok
ok
